## import

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
import warnings; warnings.filterwarnings("ignore");
from sklearn.metrics import mean_squared_error, make_scorer
import re


In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

## load dataset

In [3]:
stemmer = SnowballStemmer('english')
df_train = pd.read_csv('/Users/zhangyiman/Desktop/train.csv',encoding="ISO-8859-1")
df_test = pd.read_csv('/Users/zhangyiman/Desktop/test.csv',encoding="ISO-8859-1")
df_attr = pd.read_csv('/Users/zhangyiman/Desktop/attributes.csv')
df_pro_desc = pd.read_csv('/Users/zhangyiman/Desktop/product_descriptions.csv',encoding="ISO-8859-1")

num_train = df_train.shape[0]

## functions

In [4]:
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

In [5]:
def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_) 
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [6]:
class regvalues(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hidden):
        dropcols=['id','relevance','search_term','product_title','product_description','product_info','attr','brand']
        hidden = hidden.drop(dropcols,axis=1).values
        return hidden
    
class regvalues1(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hidden):
        dropcols=['id','relevance','product_uid','search_term','product_title','product_description','product_info','attr','brand']
        hidden = hidden.drop(dropcols,axis=1).values
        return hidden

class txtcolumn(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

In [7]:
collection = list(df_attr.name)
import re
from fuzzyfinder import fuzzyfinder
#suggestions = fuzzyfinder('material', L)
def fuzzyfinder(user_input, collection):
        suggestions =[]
        pattern = '.*'.join(user_input) # Converts 'djm' to 'd.*j.*m'
        regex = re.compile(pattern)     # Compiles a regex.
        for item in collection:
            match = regex.search(str(item))  # Checks if the current item matches the regex.
            if match:
                #suggestions.append((match.start(), item))
                suggestions.append(item)
        return suggestions

In [8]:
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

# Extract new features from attr.csv

### brand

In [9]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
df_brand[0:9]

,product_uid,brand
9,100001.0,Simpson Strong-Tie
37,100002.0,BEHR Premium Textured DeckOver
69,100003.0,STERLING
93,100004.0,Grape Solar
122,100005.0,Delta
163,100006.0,Whirlpool
204,100007.0,Lithonia Lighting
236,100008.0,Teks
256,100009.0,House of Fara


### Style 

In [10]:
style_lower = fuzzyfinder('product style', collection)
style_upper = fuzzyfinder('Product Style', collection)
style_lower.extend(style_upper)

df_styles = df_attr[df_attr['name'].isin(style_lower)][["product_uid", "value"]].rename(columns={"value": "style"})
df_styles=df_styles.drop_duplicates(subset=['product_uid'], keep='first')
df_styles[1:5]

,product_uid,style
1784,100075.0,Cottage
2290,100092.0,Cottage
2936,100117.0,Classic
6259,100262.0,Modern


### colour 

In [11]:

colour_lower = fuzzyfinder('Color Family', collection)
colour_upper = fuzzyfinder('Color Family', collection)
colour_lower.extend(colour_upper)

df_colours = df_attr[df_attr['name'].isin(colour_lower)][["product_uid", "value"]].rename(columns={"value": "colour"})
df_colours=df_colours.drop_duplicates(subset=['product_uid'], keep='first')
#df_colours[1:5]

### type

In [12]:
df_type = fuzzyfinder('Type', collection)
df_type_= fuzzyfinder('type', collection)
df_type.extend(df_type_)

In [13]:
df_types = df_attr[df_attr['name'].isin(df_type)][["product_uid", "name"]].rename(columns={"name": "type"})
df_types=df_types.drop_duplicates(subset=['product_uid'], keep='first')
df_types[1:10]

,product_uid,type
66,100003.0,Installation Type
90,100004.0,Electrical Product Type
107,100005.0,Bath Faucet Type
132,100006.0,Appliance Type
180,100007.0,Battery Power Type
227,100008.0,Fastener Thread Type
259,100009.0,Moulding Type
281,100010.0,Landscape Supply Type
309,100011.0,Drive type


### shape

In [14]:

shape_lower = fuzzyfinder('shape', collection)
shape_upper = fuzzyfinder('Shape', collection)
shape_lower.extend(shape_upper)

df_shapes = df_attr[df_attr['name'].isin(shape_lower)][["product_uid", "value"]].rename(columns={"value": "shape"})
df_shapes=df_shapes.drop_duplicates(subset=['product_uid'], keep='first')
df_shapes[1:5]

,product_uid,shape
130,100005.0,Fixed Mount
227,100008.0,Standard
444,100016.0,Rectangular
666,100026.0,Square


# Merge data

In [15]:
num_train = df_train.shape[0] #74067
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True) #train+test zongxiangdiejia 240760
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid') #240760
df_all = pd.merge(df_all, df_styles, how='left', on='product_uid')
#df_all = pd.merge(df_all, df_function, how='left', on='product_uid')
df_all = pd.merge(df_all, df_colours, how='left', on='product_uid')
df_all = pd.merge(df_all, df_types, how='left', on='product_uid')
df_all = pd.merge(df_all, df_shapes, how='left', on='product_uid')
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style,colour,type,shape
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,NaN,NaN,NaN,NaN
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,NaN,NaN,NaN,NaN
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver,Cottage,Browns / Tans,Paint Product Type,NaN
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...,Delta,NaN,Chrome,Bath Faucet Type,Fixed Mount
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...,Delta,NaN,Chrome,Bath Faucet Type,Fixed Mount


# Need to process new feature

In [16]:
df_all['shape'] = df_all['shape'].map(lambda x:str_stem(x)) 
df_all['type'] = df_all['type'].map(lambda x:str_stem(x)) 
df_all['colour'] = df_all['colour'].map(lambda x:str_stem(x)) 
df_all['style'] = df_all['style'].map(lambda x:str_stem(x)) 
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x)) 
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['len_of_search'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['percentage_title'] = df_all['word_in_title']/df_all['len_of_search']
df_all['percentage_description'] = df_all['word_in_description']/df_all['len_of_search']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']+"\t"+df_all['type']
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_type'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['percentage_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_brand = pd.unique(df_all.brand.ravel()) 


In [17]:
d={}
i = 1000
for s in df_brand:
    #print(s)
    d[s]=i
    i+=3
#print(d)
df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x)) #len(x): include the space
df_all[0:2]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style,colour,type,...,word_in_title,word_in_description,percentage_title,percentage_description,attr,word_in_brand,word_in_type,percentage_brand,brand_feature,search_term_feature
0,2,simpson strong tie 12 gaug angl,100001,3.0,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,1,1,0.5,0.5,angl bracket\tsimpson strong tie\tnull,0,0,0.0,1000,12
1,3,simpson strong tie 12 gaug angl,100001,2.5,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,1,1,0.5,0.5,l bracket\tsimpson strong tie\tnull,0,1,0.0,1000,9


## encoding

In [18]:
from sklearn import preprocessing
#Set the random seed
np.random.seed(12)
# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

#encode feature
df_all["style"] = label_encoder.fit_transform(df_all["style"])
df_all["colour"] = label_encoder.fit_transform(df_all["colour"])
df_all["type"] = label_encoder.fit_transform(df_all["type"])
df_all["shape"] = label_encoder.fit_transform(df_all["shape"])

## X_train, X_test, Y_train, Y_test

In [19]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values

X_train = df_train[:]
#X_train = df_train.drop(['id','relevance'],axis=1).values

X_test = df_test[:]
#X_test = df_test.drop(['id','relevance'],axis=1).values
#X_train = df_train[:]
#X_test = df_test[:]

In [20]:
X_train[1:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style,colour,type,...,word_in_title,word_in_description,percentage_title,percentage_description,attr,word_in_brand,word_in_type,percentage_brand,brand_feature,search_term_feature
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,5,77,303,...,1,1,0.500000,0.500000,l bracket\tsimpson strong tie\tnull,0,1,0.00,1000,9
2,9,behr premium textur deckov 1gal. #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,2,27,315,...,1,1,0.500000,0.500000,deck over\tbehr premium textur deckov\tpaint p...,1,0,0.25,1003,9
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,5,45,39,...,1,1,0.333333,0.333333,rain shower head\tdelta\tbath faucet type,0,0,0.00,1006,16
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,5,45,39,...,3,3,1.000000,1.000000,shower onli faucet\tdelta\tbath faucet type,0,1,0.00,1006,18


# train the model

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
#from sklearn import pileline
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
rf = RandomForestRegressor(n_estimators = 730, n_jobs = -1, random_state = 2120, verbose = 1)
TFIDF = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')    # sublinear_tf=True, use_idf=True
TSVD = TruncatedSVD(n_components=10, random_state = 2016)
clf = Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  regvalues()),  
                        ('txt1', Pipeline([('s1', txtcolumn(key='search_term')), ('tfidf1', TFIDF), ('tsvd1', TSVD)])),
                        ('txt2', Pipeline([('s2', txtcolumn(key='product_title')), ('tfidf2', TFIDF), ('tsvd2', TSVD)])),
                        ('txt3', Pipeline([('s3', txtcolumn(key='product_description')), ('tfidf3', TFIDF), ('tsvd3', TSVD)])),
                        ('txt4', Pipeline([('s4', txtcolumn(key='brand')), ('tfidf4', TFIDF), ('tsvd4', TSVD)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.05,
                        'txt2': 0.05,
                        'txt3': 0.05,
                        'txt4': 0.1
                        },
                #n_jobs = -1
                              )
        ), ('rf', rf)   ])
param_grid = {'rf__max_features': [8], 'rf__max_depth': [18]}
#param_grid = {'rf__max_features': [8], 'rf__max_depth': [18], 'rf__n_estimators':[732,733],'rf__random_state':[2115,2120,2125,2130]}
#param_grid = {'rf__max_features': [8], 'rf__max_depth': [18], 'rf__random_state':[2016,1900,2100]}
#param_grid = {'rf__max_features': [8], 'rf__max_depth': [18],'rf__n_estimators':[725,727,730,733]}
#param_grid = {'rf__max_features': [8], 'rf__max_depth': [18]}
rfbest = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = -1, cv = 2, verbose = 20,scoring='neg_mean_squared_error')
rfbest.fit(X_train, y_train)
y_predrf = rfbest.predict(X_test)
rfbest.best_params_

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] rf__max_features=8, rf__max_depth=18 ............................
[CV] rf__max_features=8, rf__max_depth=18 ............................


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done 730 out of 730 | elapsed:   55.7s finished
[Parallel(n_jobs=-1)]: Done 730 out of 730 | elapsed:   56.4s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 730 out of 730 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 730 out of 730 | elapse

[CV]  rf__max_features=8, rf__max_depth=18, score=-0.218606, total= 1.3min


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 730 out of 730 | elapsed:    1.2s finished


[CV]  rf__max_features=8, rf__max_depth=18, score=-0.224390, total= 1.3min


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.5min finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 730 out of 730 | elapsed:   56.8s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done 730 out of 730 | elapsed:    4.6s finished


{'rf__max_depth': 18, 'rf__max_features': 8}

In [22]:
from sklearn import pipeline,grid_search
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(hidden_layer_sizes=(10, 5),  alpha=0.0001, random_state=2050, batch_size=171)
TFIDF1 = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')    # sublinear_tf=True, use_idf=True
TSVD1 = TruncatedSVD(n_components=10, random_state = 2016)
nnclf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  regvalues1()),  
                        ('txt1', pipeline.Pipeline([('s1', txtcolumn(key='search_term')), ('tfidf1', TFIDF1), ('tsvd1', TSVD1)])),
                        ('txt2', pipeline.Pipeline([('s2', txtcolumn(key='product_title')), ('tfidf2', TFIDF1), ('tsvd2', TSVD1)])),
                        ('txt3', pipeline.Pipeline([('s3', txtcolumn(key='product_description')), ('tfidf3', TFIDF1), ('tsvd3', TSVD1)])),
                        ('txt4', pipeline.Pipeline([('s4', txtcolumn(key='brand')), ('tfidf4', TFIDF1), ('tsvd4', TSVD1)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.05,
                        'txt2': 0.05,
                        'txt3': 0.05,
                        'txt4': 0.01
                        },
                #n_jobs = -1
                              )
        ), ('nn', nn)   ])
#nnparam_grid = {'nn__hidden_layer_sizes':[(10,3),(10,5)], 'nn__alpha':[1e-5, 1e-3]} #add more
#nnparam_grid = {'nn__hidden_layer_sizes':[(10,3),(10,5)], 'nn__alpha':[1e-4, 1e-5,1e-6], 'nn__batch_size':[150,160,170,180]} #add more
nnparam_grid = {'nn__hidden_layer_sizes':[(10,5)], 'nn__alpha':[1e-4, 1e-5]}
#nnparam_grid = {'nn__hidden_layer_sizes':[(10,3),(10,5)], 'nn__alpha':[1e-4,1e-5],'nn__batch_size':[167,168,169,170,171,172,173],'nn__random_state':[2000,2050,2100,2150,2200]}

nnbest = grid_search.GridSearchCV(estimator = nnclf, param_grid = nnparam_grid, n_jobs = -1, cv = 2, verbose = 20, scoring='neg_mean_squared_error')
nnbest.fit(X_train, y_train)
y_prednn = nnbest.predict(X_test)
nnbest.best_params_

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] nn__alpha=0.0001, nn__hidden_layer_sizes=(10, 5) ................
[CV] nn__alpha=0.0001, nn__hidden_layer_sizes=(10, 5) ................
[CV] nn__alpha=1e-05, nn__hidden_layer_sizes=(10, 5) .................
[CV] nn__alpha=1e-05, nn__hidden_layer_sizes=(10, 5) .................
[CV]  nn__alpha=0.0001, nn__hidden_layer_sizes=(10, 5), score=-0.252056 -  30.2s


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   34.1s


[CV]  nn__alpha=0.0001, nn__hidden_layer_sizes=(10, 5), score=-0.251428 -  28.7s


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   35.0s remaining:   35.0s


[CV]  nn__alpha=1e-05, nn__hidden_layer_sizes=(10, 5), score=-0.251369 -  29.6s
[CV]  nn__alpha=1e-05, nn__hidden_layer_sizes=(10, 5), score=-0.255670 -  27.4s


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   39.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   39.1s finished


{'nn__alpha': 0.0001, 'nn__hidden_layer_sizes': (10, 5)}

# evaluation

In [23]:
y_pred = 0.9*y_predrf +0.1*y_prednn
df_sol = pd.read_csv('/Users/zhangyiman/Desktop/solution.csv', encoding="ISO-8859-1")
df_sol['pred'] = y_pred
df_fliter = df_sol[df_sol.Usage == "Public"]

In [24]:
from sklearn.metrics import mean_squared_error, make_scorer
mse = mean_squared_error(df_fliter.relevance, df_fliter.pred, sample_weight=None, multioutput='uniform_average')
np.sqrt(mse)

0.47154933331984566

In [25]:
from sklearn.metrics import r2_score
R2 = r2_score(df_fliter.relevance, df_fliter.pred)  
print(R2)

0.224835163548


In [26]:
#from sklearn.model_selection import cross_val_score
#score_rf = cross_val_score(clf,X_train,y_train,cv=5,scoring='mean_squared_error')
#score_nn = cross_val_score(nn,X_train,y_train,cv=5,scoring='mean_squared_error')
#score = 0.5*(score_rf + score_nn)
#score.mean()